In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data
from sklearn.neighbors import kneighbors_graph


In [2]:
# Load iris dataset
iris = load_iris()
scaler = StandardScaler()
X = torch.tensor(scaler.fit_transform(iris.data), dtype=torch.float)
y = torch.tensor(iris.target, dtype=torch.long)
print("Data Shape: ", X.shape)
print("Target Shape: ", y.shape)


Data Shape:  torch.Size([150, 4])
Target Shape:  torch.Size([150])


In [3]:
torch.unique(y)


tensor([0, 1, 2])

In [4]:
# Create graph and edge indices
knn_graph = kneighbors_graph(X, n_neighbors=5, mode='connectivity', include_self=True)
edge_index = torch.tensor(knn_graph.nonzero(), dtype=torch.long)


/tmp/ipykernel_22143/3847851584.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  edge_index = torch.tensor(knn_graph.nonzero(), dtype=torch.long)


In [5]:
# Create Pytorch Geometric Data Object
data = Data(x=X, edge_index=edge_index, y=y)


In [6]:
data.x.shape

torch.Size([150, 4])

In [7]:
data.edge_index.size(0)

2

In [8]:
X.size(1)


4

In [9]:
# Train/Test splitting
train_mask = torch.zeros(X.size(0), dtype=torch.bool)
test_mask = torch.zeros(X.size(0), dtype=torch.bool)

train_idx, test_idx = train_test_split(range(X.size(0)), test_size=0.3, stratify=y)
train_mask[train_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.test_mask = test_mask


In [10]:
data

Data(x=[150, 4], edge_index=[2, 750], y=[150], train_mask=[150], test_mask=[150])

In [11]:
# Define GCN Model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels[0])
        self.conv2 = GCNConv(hidden_channels[0], hidden_channels[1])
        self.conv3 = GCNConv(hidden_channels[1], out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x


In [12]:
# Set up Model, Optmizer and Loss function
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = GCN(in_channels=4, hidden_channels=[16, 32], out_channels=3).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


In [13]:
# Training Loop
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} --> Loss: {loss.item():.4f}")


Epoch: 0 --> Loss: 1.4021
Epoch: 10 --> Loss: 0.4173
Epoch: 20 --> Loss: 0.2711
Epoch: 30 --> Loss: 0.2043
Epoch: 40 --> Loss: 0.1486
Epoch: 50 --> Loss: 0.1159
Epoch: 60 --> Loss: 0.1071
Epoch: 70 --> Loss: 0.1027
Epoch: 80 --> Loss: 0.0990
Epoch: 90 --> Loss: 0.0968
Epoch: 100 --> Loss: 0.0951
Epoch: 110 --> Loss: 0.0935
Epoch: 120 --> Loss: 0.0920
Epoch: 130 --> Loss: 0.0904
Epoch: 140 --> Loss: 0.0889
Epoch: 150 --> Loss: 0.0874
Epoch: 160 --> Loss: 0.0860
Epoch: 170 --> Loss: 0.0845
Epoch: 180 --> Loss: 0.0830
Epoch: 190 --> Loss: 0.0815


In [14]:
# Evaluate model
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
correct = pred[data.test_mask] == data.y[data.test_mask]
acc = int(correct.sum()) / int(data.test_mask.sum())
print(f"Test Accuracy: {acc:.4f}")


Test Accuracy: 1.0000
